<a href="https://colab.research.google.com/github/Anubhav2017/Genomics_notebooks/blob/main/approach_kmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D,Reshape, LSTM, Dropout, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.utils import shuffle

bases={'A':np.array([0,0,0,1]), 'C':np.array([0,0,1,0]), 'G':np.array([0,1,0,0]), 'T':np.array([1,0,0,0])}

def Kmers_funct(seq, size):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

def one_hot_encode_2(y,num_classes):
    y_encoded=[]
    for value in y:
	    letter = [0 for _ in range(num_classes)]
	    letter[value] = 1
	    y_encoded.append(letter)
    return np.array(y_encoded,dtype=np.float16)



In [ ]:
import random

def selectref(el,pa,pc,pg,pt):

  dicref={'U':['T'],
  'R':['A','G'],
  'Y':['C','T'],
  'S':['G','C'],
  'W':['A','T'],
  'K':['G','T'],
  'M':['A','C'],
  'B':['C','G','T'],
  'D':['A','G','T'],
  'H':['A','C','T'],
  'V':['A','C','G'],
  'N':['A','T','G','C'],
  }

  dicprobs={'U':[1],
  'R':[pa/(pa+pg),pg/(pa+pg)],
  'Y':[pc/(pc+pt),pt/(pc+pt)],
  'S':[pg/(pc+pg),pc/(pc+pg)],
  'W':[pa/(pa+pt),pt/(pa+pt)],
  'K':[pg/(pt+pg),pt/(pt+pg)],
  'M':[pa/(pa+pc),pc/(pa+pc)],
  'B':[pc/(pc+pg+pt),pg/(pc+pg+pt),pt/(pc+pg+pt)],
  'D':[pa/(pa+pg+pt),pg/(pa+pg+pt),pt/(pa+pg+pt)],
  'H':[pa/(pc+pa+pt),pc/(pc+pa+pt),pt/(pc+pa+pt)],
  'V':[pa/(pc+pg+pa),pc/(pc+pg+pa),pg/(pc+pg+pa)],
  'N':[pa/(pc+pg+pa+pt),pt/(pc+pg+pa+pt),pg/(pc+pg+pa+pt),pc/(pc+pg+pa+pt)]}

  # print(dicprobs[el])
  
  return np.random.choice(dicref[el],p=dicprobs[el])
  # return np.random.choice(dicref[el],p=dicprobs[el])





In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !pip install BioPython

In [ ]:
from Bio import SeqIO
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

unique_elems=dict()

all_sequences=[]

for sequence in SeqIO.parse('ncbi_16s_18s_merged.fasta', "fasta"):
    unique_elems[sequence.description.split()[1]]=0
    seq=""

    base_count=dict()
    base_count['A']=0
    base_count['C']=0
    base_count['G']=0
    base_count['T']=0
    base_count['others']=0

    for el in sequence.seq:
      if el not in base_count.keys():

        base_count['others']+=1
      else:
        base_count[el]+=1
    
    na=base_count['A']
    nc=base_count['C']
    ng=base_count['G']
    nt=base_count['T']

    pa=float(na/(na+nc+ng+nt))
    pc=float(nc/(na+nc+ng+nt))
    pg=float(ng/(na+nc+ng+nt))
    pt=float(nt/(na+nc+ng+nt))

    
    for el in sequence.seq:
      if el not in base_count.keys():

        seq += (selectref(el,pa,pc,pg,pt))
      else:
        seq+=el
    
    words = Kmers_funct(seq, size=6)
    joined_sentence = ' '.join(words)
    all_sequences.append(joined_sentence)

X=cv.fit_transform(all_sequences).toarray()

print(X.shape)



    
    

    

In [ ]:
# from Bio import SeqIO
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

# unique_elems=dict()

# for sequence in SeqIO.parse('ncbi_16s_18s_merged.fasta', "fasta"):
#     unique_elems[sequence.description.split()[1]]=0# from Bio import SeqIO
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

# unique_elems=dict()

# for sequence in SeqIO.parse('ncbi_16s_18s_merged.fasta', "fasta"):
#     unique_elems[sequence.description.split()[1]]=0

In [ ]:
i=0
x_data=[]
y_data=[]
for el in unique_elems.keys():
    unique_elems[el]=i
    i+=1

i=0
for sequence in SeqIO.parse('ncbi_16s_18s_merged.fasta', "fasta"):
    x_data.append(X[i])
    y_data.append(unique_elems[sequence.description.split()[1]])
    i+=1

lm=len(unique_elems)

class_count=dict()

for i in range(lm):
  class_count[i]=0

for el in y_data:
  class_count[el]+=1



In [ ]:
import matplotlib.pyplot as plt

plt.plot(list(class_count.keys()), list(class_count.values()), color='g')
plt.title("Number of samples corresponding to various genus")

In [ ]:
dominant_classes=([idx for idx, element in enumerate(class_count.values()) if element>50])


indices_to_remove=[]

for el in dominant_classes:
  # print(list(np.where(np.array(y_data) == el))[0])
  indices_to_remove+=random.sample(list(np.where(np.array(y_data) == el)[0]),class_count[el]-50)
  



In [ ]:
x_data_new=[]
y_data_new=[]

for i in range(len(y_data)):
  if i not in indices_to_remove:
    x_data_new.append(x_data[i])
    y_data_new.append(y_data[i])



In [ ]:

def release_list(a):
   del a[:]
   del a

release_list(x_data)
release_list(y_data)

In [ ]:
class_count_new=dict()

for i in range(lm):
  class_count_new[i]=0
for el in y_data_new:
  class_count_new[el]+=1


plt.plot(list(class_count_new.keys()), list(class_count_new.values()), color='g')
plt.title("Number of samples corresponding to various genus")

In [ ]:
# desired_count=dict()

# for i in range(lm):
#   if class_count[i]<10:
#     desired_count[i]=10
#   else:
#     desired_count[i]=class_count[i]


In [ ]:
# from imblearn.over_sampling import SMOTE, RandomOverSampler

# oversample = RandomOverSampler(sampling_strategy=desired_count)
# x,y=oversample.fit_resample(x_data_new, y_data_new)

In [ ]:

# plt.bar(desired_count.keys(), desired_count.values(), color='g')
# plt.title("Number of samples corresponding to various genus")

In [ ]:
x_data=x_data_new.copy()
y_data=y_data_new.copy()

In [ ]:

x_train=[]
y_train=[]

x_test=[]
y_test=[]

rare_items=[]

for item in class_count_new.keys():
  if class_count[item] < 3:
    rare_items.append(item)

i=0
for el in y_data_new:
  if el in rare_items:
    x_train.append(x_data_new[i])
    y_train.append(el)
    x_train.append(x_data_new[i])
    y_train.append(el)
    x_test.append(x_data_new[i])
    y_test.append(el)

  i+=1


x_data,y_data=shuffle(x_data,y_data)

x_train+=x_data[:int(len(x_data)*0.8)]
y_train+=y_data[:int(len(y_data)*0.8)]


x_test+=x_data[int(len(x_data)*0.8):]
y_test+=y_data[int(len(y_data)*0.8):]

print(len(x_train))

y_train=one_hot_encode_2(y_train,len(unique_elems))
y_test=one_hot_encode_2(y_test,len(unique_elems))

x_train=np.array(x_train,dtype=np.float32)
x_train=np.reshape(x_train,(-1,4096,1))

x_test=np.array(x_test,dtype=np.float32)
x_test=np.reshape(x_test,(-1,4096,1))

In [ ]:
import os
checkpoint_path = "/content/drive/My Drive/training_genomics_kmer/{epoch:03d}/cp-{epoch:04d}"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
model=Sequential()
model.add((Conv1D(filters=6, kernel_size=6, activation='relu')))
model.add((Conv1D(filters=3, kernel_size=6, activation='relu')))
model.add(Flatten())
model.add(Dense(units=1024,activation='relu'))
model.add(Dense(units=lm, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.load_weights("/content/drive/My Drive/training_genomics_kmer/010/cp-0010")
early_stopping = EarlyStopping(monitor='loss', patience=2, mode='min')

history = model.fit(x_train[0:1], y_train[0:1], epochs=1, verbose=1)
model.summary()
history = model.fit(x_train, y_train, epochs=9, batch_size=64, validation_data= (x_test,y_test),callbacks=[cp_callback])


In [ ]:
from sklearn import metrics

preds=model.predict(x_data)

y_pred_labels=[np.argmax(el) for el in preds]
y_data_labels=[np.argmax(el) for el in y_data]

print(metrics.confusion_matrix(y_data_labels, y_pred_labels))

print(metrics.classification_report(y_data_labels, y_pred_labels, digits=3))


In [ ]:
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score

fpr = {}
tpr = {}
thresh ={}
roc_auc={}
n_class = lm
# y_true=np.array(y_true)
preds_training_data=model.predict(x_train)
# print(preds[0:3])
# print(y_test[0:3])

for i in range(n_class):
  # print(y_train[:,i])

  if len(np.unique(y_test[:,i]))==2:
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test[:,i], preds[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
print(roc_auc)

In [ ]:
release_list(y_data)

In [ ]:
import random
a=[1,2,3,4]
b=random.sample(a,2)
print(a)
print(b)